In [ ]:
# ## Importing Libraries
import os
import gc
import numpy as np
import pandas as pd
import logging
import json
import traceback
import joblib

# Machine Learning imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, accuracy_score, f1_score

from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy import stats

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau



In [ ]:
gc.collect()

In [ ]:
data_dir = os.path.join('/mnt', 'd', 'work', 'Walsh', 'Capstone', 'Published', 'Data')
prk_folder = os.path.join(data_dir, 'parquets')
models_folder = os.path.join(data_dir, 'models')

# Global variables
SAMPLING_RATES = {
    "MW": 512, "EP": 128, "MU": 220, "IN": 128,
    "EP22_Train": 128, "EP22_Test": 128
}

In [ ]:
MU_file = os.path.join(prk_folder, 'MU.parquet')
MU_Process_file = os.path.join(prk_folder, 'MU_Processed.parquet')
MU_Features_file = os.path.join(prk_folder, 'MU_Features.parquet')
MU_Process_Agg_file = os.path.join(prk_folder, 'MU_processed_aggregated.parquet')

EP_file = os.path.join(prk_folder, 'EP.parquet')
EP_Process_file = os.path.join(prk_folder, 'EP_Processed.parquet')
EP_Features_file = os.path.join(prk_folder, 'EP_Features.parquet')
EP_Process_Agg_file = os.path.join(prk_folder, 'EP_processed_aggregated.parquet')

MW_file = os.path.join(prk_folder, 'MW.parquet')
MW_Process_file = os.path.join(prk_folder, 'MW_Processed.parquet')
MW_Features_file = os.path.join(prk_folder, 'MW_Features.parquet')
MW_Process_Agg_file = os.path.join(prk_folder, 'MW_processed_aggregated.parquet')

IN_file = os.path.join(prk_folder, 'IN.parquet')
IN_Process_file = os.path.join(prk_folder, 'IN_Processed.parquet')
IN_Features_file = os.path.join(prk_folder, 'IN_Features.parquet')
IN_Process_Agg_file = os.path.join(prk_folder, 'IN_processed_aggregated.parquet')

EP22_Train_file = os.path.join(prk_folder, 'EP22_Train.parquet')
EP22_Train_Process_file = os.path.join(prk_folder, 'EP22_Train_Processed.parquet')
EP22_Train_Features_file = os.path.join(prk_folder, 'EP22_Train_Features.parquet')
EP22_Train_Process_Agg_file = os.path.join(prk_folder, 'EP22_Train_processed_aggregated.parquet')

EP22_Test_file = os.path.join(prk_folder, 'EP22_Test.parquet')
EP22_Test_Process_file = os.path.join(prk_folder, 'EP22_Test_Processed.parquet')
EP22_Test_Features_file = os.path.join(prk_folder, 'EP22_Test_Features.parquet')
EP22_Test_Process_Agg_file = os.path.join(prk_folder, 'EP22_Test_processed_aggregated.parquet')

In [ ]:
def aggregate_data(df, interested_col = 'data'):

    new_row = {}
    tmp = {}
    try:
        tmp_gpr_df = df.groupby(['device', 'event'])
        if interested_col in df.columns:
            if interested_col != 'data':
                df.columns = df.columns.str.replace(interested_col, 'data')
            for grp in tmp_gpr_df:
                tmp = {}
                if grp[1]["event"].empty:
                    continue
                for index, row in grp[1].iterrows():
                    tmp[row['channel']] = row['data']
                
                tmp["code"] = int(grp[1]["code"].values[0])
                new_row[int(grp[1]["event"].values[0])] = tmp
    except Exception as e:
        print(f"Error processing group: {e}")
        print(traceback.format_exc())

    result_df = pd.DataFrame.from_dict(new_row, orient='index')
    result_df.reset_index(inplace=True)

    return result_df

In [ ]:
mw_process_df = pd.read_parquet(MW_Process_file)
mw_agg_df = aggregate_data(mw_process_df, 'processed_signal')
mw_agg_df.to_parquet(MW_Process_Agg_file, index=False)
mw_process_df = None
mw_agg_df = None
gc.collect()


In [ ]:
mu_process_df = pd.read_parquet(MU_Process_file)
mu_agg_df = aggregate_data(mu_process_df, 'processed_signal')
mu_agg_df.to_parquet(MU_Process_Agg_file, index=False)
mu_process_df = None
mu_agg_df = None
gc.collect()

In [ ]:
in_process_df = pd.read_parquet(IN_Process_file)
in_agg_df = aggregate_data(in_process_df, 'processed_signal')
in_agg_df.to_parquet(IN_Process_Agg_file, index=False)
in_process_df = None
in_agg_df = None
gc.collect()

In [ ]:
ep_process_df = pd.read_parquet(EP_Process_file)
ep_agg_df = aggregate_data(ep_process_df, 'processed_signal')
ep_agg_df.to_parquet(IN_Process_Agg_file, index=False)
ep_process_df = None
ep_agg_df = None
gc.collect()

In [ ]:
def prepare_column_for_training(processed_df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Converts the specified column (time-series data) into a feature matrix.
    Each time point in the signal becomes a separate feature.
    """
    if col_name not in processed_df.columns or 'code' not in processed_df.columns:
        logging.error(f"Input DataFrame must contain '{col_name}' and 'code' columns.")
        return pd.DataFrame()

    # Removing Noise signals
    processed_df = processed_df[processed_df["code"] != -1]

    # More robust filter that works for both lists and ndarrays.
    df = processed_df[processed_df[col_name].apply(lambda x: hasattr(x, '__len__') and len(x) > 0)].copy()

    if df.empty:
        logging.warning("No valid signals found in the processed DataFrame to prepare for training.")
        return pd.DataFrame()

    # Use np.vstack for robust creation of the feature matrix from lists or arrays.
    signal_matrix = np.vstack(df[col_name].values)
    
    # Create new column names for each time point (feature)
    feature_names = [f'data_point_{i}' for i in range(signal_matrix.shape[1])]
    
    # Create a new DataFrame for the features
    features_df = pd.DataFrame(signal_matrix, columns=feature_names, index=df.index)
    
    # Combine the new features DataFrame with the essential 'code' column from the original df
    model_ready_df = pd.concat([df[['code']], features_df], axis=1)
    
    return model_ready_df

In [ ]:
# def prepare_single_channelfor_training1(df, label_col='code', test_size=0.2, random_state=42):
#     feature_cols = [col for col in df.columns if col.startswith('data_point_')]
    
#     if not feature_cols:
#         logging.error("No feature columns (starting with 'data_point_') found in the DataFrame for training.")
#         return None, np.array([]), np.array([]), np.array([]), np.array([])

#     X = df[feature_cols].copy().fillna(0)
#     y = df[label_col].copy()
    
#     if X.empty or y.empty:
#         logging.error("Feature matrix (X) or label vector (y) is empty.")
#         return None, np.array([]), np.array([]), np.array([]), np.array([])

#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)
    
#     if len(np.unique(y)) > 1:
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)
#         X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test)
#         return scaler, X_train, y_train, X_val, y_val, X_test, y_test
#     else:
#         logging.warning("Only one class present in data. Cannot perform stratified split.")
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
#         X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test)
#         return scaler, X_train, y_train, X_val, y_val, X_test, y_test
    



In [ ]:
def prepare_single_channel_for_training(df, label_col='code', col_name = None, test_size=0.2, random_state=42):
    df = df[df['code'] != -1]  # Removing Noise signals

    if "index" in df.columns:
        df = df.drop(columns=["index"])

    id_vars = ['code']

    if col_name == 'None' or col_name not in df.columns:
        logging.error(f"Column name for signal data is not provided or does not exist in DataFrame.")
        return None, np.array([]), np.array([]), np.array([]), np.array([])
    
    if label_col not in df.columns:
        logging.error(f"Label column '{label_col}' does not exist in DataFrame.")
        return None, np.array([]), np.array([]), np.array([]), np.array([])
    
    # All other columns are assumed to be the channel data columns.
    channel_cols = [col_name]

    concatenated_arrays = df[channel_cols].apply(
        lambda row: np.concatenate(row.values), axis=1
    )

    expanded_df = pd.DataFrame(
        concatenated_arrays.to_list(),
        index=df.index
    )
    
    # Convert all column names to string format
    expanded_df.columns = [str(col) for col in expanded_df.columns]
    complete_df = pd.concat([df[id_vars], expanded_df], axis=1)

    if complete_df.empty:
        logging.error("No feature columns ( found in the DataFrame for training.")
        return None, np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([])

    X = complete_df
    y = df[label_col].copy()
    
    if X.empty or y.empty:
        logging.error("Feature matrix (X) or label vector (y) is empty.")
        return None, np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([])

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    if len(np.unique(y)) > 1:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)
        X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test)
        return scaler, X_train, y_train, X_val, y_val, X_test, y_test
    else:
        logging.warning("Only one class present in data. Cannot perform stratified split.")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
        X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test)
        return scaler, X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
def prepare_multi_channelfor_training(df, label_col='code', test_size=0.2, random_state=42):

    df = df[df['code'] != -1]  # Removing Noise signals

    # 'index' and 'code' are the identifier columns.
    id_vars = ['index', 'code']


    # All other columns are assumed to be the channel data columns.
    channel_cols = [col for col in df.columns if col not in id_vars]

    concatenated_arrays = df[channel_cols].apply(
        lambda row: np.concatenate(row.values), axis=1
    )

    expanded_df = pd.DataFrame(
        concatenated_arrays.to_list(),
        index=df.index
    )
    
    # Convert all column names to string format
    expanded_df.columns = [str(col) for col in expanded_df.columns]
    complete_df = pd.concat([df[id_vars], expanded_df], axis=1)

    if "index" in complete_df.columns:
        complete_df = complete_df.drop(columns=["index"])


    # print(f"Complete DataFrame shape: {complete_df.shape}")
    # print(f"Complete DataFrame columns: {complete_df.columns.tolist()}")
    if complete_df.empty:
        logging.error("No feature columns (starting with 'data_point_') found in the DataFrame for training.")
        return None, np.array([]), np.array([]), np.array([]), np.array([])

    X = complete_df
    y = df[label_col].copy()
    
    if X.empty or y.empty:
        logging.error("Feature matrix (X) or label vector (y) is empty.")
        return None, np.array([]), np.array([]), np.array([]), np.array([])

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    if len(np.unique(y)) > 1:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)
        X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test)
        return scaler, X_train, y_train, X_val, y_val, X_test, y_test
    else:
        logging.warning("Only one class present in data. Cannot perform stratified split.")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
        X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test)
        return scaler, X_train, y_train, X_val, y_val, X_test, y_test
    


In [ ]:
gc.collect()

In [ ]:
mw_agg_df = pd.read_parquet(MW_Process_Agg_file)
mu_agg_df = pd.read_parquet(MU_Process_Agg_file)
in_agg_df = pd.read_parquet(IN_Process_Agg_file)
ep_agg_df = pd.read_parquet(EP_Process_Agg_file)

In [ ]:
scaler, X_train, y_train, X_val, y_val, X_test, y_test = prepare_multi_channelfor_training(mu_agg_df)
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
mu_model_df = prepare_column_for_training(mu_agg_df, 'FP2')
mu_model_df

In [ ]:
def evaluate_RandomForest_model(X_train, y_train, X_val, y_val, X_test, y_test,  Data_name: str, Model_name : str, n_estimators=100, max_depth=None, random_state=42):
    print("Training Random Forest Classifier...")
    
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    model.fit(X_train, y_train)
    
    # Validation
    y_val_pred = model.predict(X_test)
    val_accuracy = accuracy_score(y_test, y_val_pred)

    val_f1 = f1_score(y_test, y_val_pred, average='weighted')

    report = classification_report(y_test, y_val_pred, zero_division=0)

    print(f"Validation Accuracy: {val_accuracy:.4f}, F1 Score: {val_f1:.4f}")

    # Save model
    model_path = os.path.join(models_folder, f"{Data_name}_{Model_name}_RF_model.joblib")
    joblib.dump(model, model_path)
    print(f"Model saved to {model_path}")
    
    # Save Accuracy and Report
    acc_report_path = os.path.join(models_folder, f"{Data_name}_{Model_name}_RF_report.txt")
    with open(acc_report_path, 'w') as f:
        f.write(f"Test Accuracy: {val_accuracy:.4f}\n")
        f.write(f"Classification Report:\n{report}\n")
    
    return model, val_accuracy, val_f1

In [ ]:
def BasicSingleChannelCNN(n_timesteps, n_features, n_outputs):
    model = Sequential([
        Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(n_features, n_timesteps)),
        MaxPooling1D(pool_size=2),
        Dropout(0.5),
        Conv1D(filters=64, kernel_size=5, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.5),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(n_outputs, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = BasicSingleChannelCNN(256, 2214, 10)
model.summary()

In [ ]:
m_layers = model.layers

for layer in m_layers:
    print(f"Layer: {layer.name}, {type(layer)}")

In [ ]:
def evaluate_single_channel_dnn_models(X_train, y_train, X_val, y_val, X_test, y_test, 
                        Data_name: str, Model_name : str, 
                        num_classes=10, models_folder=models_folder):
    """Train and evaluate selected deep model.
    Expects X_* as (samples, features) for BasicDNN and (samples, n_features, n_timesteps) for CNN models.
    y_* are integer class labels.
    """
    if X_train is None or getattr(X_train, 'size', 0) == 0:
        print("--- DNN ---\nSkipping training: No data available.")
        return None, 0, ""

    # Ensure numpy arrays
    X_train = np.asarray(X_train)
    X_val = np.asarray(X_val)
    X_test = np.asarray(X_test)
    y_train = np.asarray(y_train)
    y_val = np.asarray(y_val)
    y_test = np.asarray(y_test)

    n_classes = int(max(y_train.max(), y_val.max(), y_test.max()) + 1)

    # One-hot encode labels
    y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes=n_classes)
    y_val_cat = tf.keras.utils.to_categorical(y_val, num_classes=n_classes)
    y_test_cat = tf.keras.utils.to_categorical(y_test, num_classes=n_classes)

    # Determine input dimensions
    is_cnn = Model_name in ['BasicSingleChannelCNN']

    if is_cnn:
        # Expect raw shape (samples, features); reshape to (samples, n_features, 1) treating each feature as timestep
        if X_train.ndim == 2:
            X_train_c = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
            X_val_c = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
            X_test_c = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
        else:
            X_train_c, X_val_c, X_test_c = X_train, X_val, X_test
        n_features, n_timesteps = X_train_c.shape[1], X_train_c.shape[2]
    else:
        n_features = X_train.shape[1]
        n_timesteps = 1  # placeholder

    # Select model
    if Model_name == 'BasicSingleChannelCNN':
        model = BasicSingleChannelCNN(n_timesteps=n_timesteps, n_features=n_features, n_outputs=n_classes)
    # elif Model_name == 'OneD_CNN_CausalDilated':
    #     model = OneD_CNN_CausalDilated(n_timesteps=n_timesteps, n_features=n_features, n_outputs=n_classes)
    else:
        print(f"Model {Model_name} not recognized.")
        return None, 0, ""

    # model.summary()

    # Callbacks
    callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    callback_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

    os.makedirs(models_folder, exist_ok=True)
    checkpointPath = os.path.join(models_folder, f"{Data_name}_best_{Model_name}_weights.keras")
    checkpointer = ModelCheckpoint(filepath=checkpointPath, verbose=0, save_best_only=True, monitor='val_loss')

    epochs, batch_size = 200, 128

    if is_cnn:
        history = model.fit(X_train_c, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=0,
                            validation_data=(X_val_c, y_val_cat), callbacks=[callback_es, callback_lr, checkpointer])
        eval_inputs = X_test_c
    else:
        history = model.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=0,
                            validation_data=(X_val, y_val_cat), callbacks=[callback_es, callback_lr, checkpointer])
        eval_inputs = X_test

    loss, accuracy = model.evaluate(eval_inputs, y_test_cat, batch_size=batch_size, verbose=0)
    y_pred_prob = model.predict(eval_inputs, batch_size=batch_size, verbose=0)
    y_pred = np.argmax(y_pred_prob, axis=1)

    report = classification_report(y_test, y_pred, zero_division=0)
    # print(f"Test Accuracy: {accuracy:.4f}\nClassification Report:\n{report}")

    # Save model
    model_save_path = os.path.join(models_folder, f"{Data_name}_{Model_name}_CNN.keras")
    model.save(model_save_path)

    # Save Accuracy and Report
    acc_report_path = os.path.join(models_folder, f"{Data_name}_{Model_name}_CNN_report.txt")
    with open(acc_report_path, 'w') as f:
        f.write(f"Test Accuracy: {accuracy:.4f}\n")
        f.write(f"Classification Report:\n{report}\n")

    print(f"Test Accuracy: {accuracy:.4f}")

    val_accuracy = accuracy_score(y_test, y_pred)
    val_f1 = f1_score(y_test, y_pred, average='weighted')

    return model, val_accuracy, val_f1

In [ ]:
mu_model_df = prepare_column_for_training(mw_agg_df, 'FP1')
scaler, X_train, y_train, X_val, y_val, X_test, y_test = prepare_for_training(mu_model_df)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

In [ ]:
model, val_accuracy, val_f1 = evaluate_dnn_models(
    X_train, y_train, X_val, y_val, X_test, y_test,
    Data_name='MW_FP1', Model_name='BasicCNN'
)
print('Final Accuracy:', val_accuracy, 'Final F1:', val_f1)

In [ ]:
model, val_accuracy, val_f1 = evaluate_dnn_models(
    X_train, y_train, X_val, y_val, X_test, y_test,
    Data_name='MW_FP1', Model_name='OneD_CNN_CausalDilated'
)
print('Final Accuracy:', val_accuracy, 'Final F1:', val_f1)

In [ ]:
mu_model_df = prepare_column_for_training(mu_agg_df, 'FP2')
scaler, X_train, y_train, X_val, y_val, X_test, y_test = prepare_for_training(mu_model_df)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)
model, val_accuracy, val_f1 = evaluate_dnn_models(
    X_train, y_train, X_val, y_val, X_test, y_test,
    Data_name='MU_FP2', Model_name='BasicCNN'
)
print('Final Accuracy:', val_accuracy, 'Final F1:', val_f1)

In [ ]:
model, val_accuracy, val_f1 = evaluate_dnn_models(
    X_train, y_train, X_val, y_val, X_test, y_test,
    Data_name='MU_FP2', Model_name='OneD_CNN_CausalDilated'
)
print('Final Accuracy:', val_accuracy, 'Final F1:', val_f1)

In [ ]:
results_df = []
for device, channels in results.items():
    for channel, models in channels.items():
        results_df.append({
                "Device": device,
                "Channel": channel,
                "RF_ACC" : models["RandomForest"]["acc"],
                "CNN_ACC" : models["BasicSingleChannelCNN"]["acc"],
                "RF_F1" : models["RandomForest"]["f1"],
                "CNN_F1" : models["BasicSingleChannelCNN"]["f1"]
            })

results_df = pd.DataFrame(results_df)
results_df = results_df.sort_values(by=["Device", "Channel"]).reset_index(drop=True)
results_df.to_csv(os.path.join(models_folder, 'model_results_summary.csv'), index=False)
results_df

In [ ]:
# mw_agg_df = pd.read_parquet(MW_Process_Agg_file)
# mu_agg_df = pd.read_parquet(MU_Process_Agg_file)
# in_agg_df = pd.read_parquet(IN_Process_Agg_file)
# ep_agg_df = pd.read_parquet(EP_Process_Agg_file)

processed_aggs = {
    "MW" : mw_agg_df,
    "MU" : mu_agg_df,
    "IN" : in_agg_df,
    "EP" : ep_agg_df
}

results_full = {
    "MW": {},
    "MU": {},
    "IN": {},
    "EP": {}
}


In [ ]:
for key in processed_aggs.keys():
    print(f"Processing {key} data...")
    df = processed_aggs[key]

    scaler, X_train, y_train, X_val, y_val, X_test, y_test = prepare_multi_channelfor_training(df)
    print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
    print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

    model_rf, rf_val_accuracy, rf_val_f1 = evaluate_RandomForest_model(
        X_train, y_train, X_val, y_val, X_test, y_test,
        Data_name=f'{key}_Full', Model_name='RandomForest'
    )

    model_cnn, cnn_val_accuracy, cnn_val_f1 = evaluate_single_channel_dnn_models(
        X_train, y_train, X_val, y_val, X_test, y_test,
        Data_name=f'{key}_Full', Model_name='BasicSingleChannelCNN'
    )



    results_full[key] = {}
    results_full[key]["RandomForest"] = {"acc": rf_val_accuracy, "f1": rf_val_f1}
    results_full[key]["BasicSingleChannelCNN"] = {"acc": cnn_val_accuracy, "f1": cnn_val_f1}
results_full

In [ ]:
results_full_data = []
for key in results_full.keys():
    print(f"Results for {key}:")
    results_full_data.append({
            "Device" : key,
            "RF_acc" : results_full[key]["RandomForest"]["acc"],
            "RF_f1" : results_full[key]["RandomForest"]["f1"],
            "CNN_acc" : results_full[key]["BasicSingleChannelCNN"]["acc"],
            "CNN_f1" : results_full[key]["BasicSingleChannelCNN"]["f1"]
        })
results_full_data = pd.DataFrame(results_full_data)
results_full_data.to_csv(os.path.join(models_folder, 'full_model_results_summary.csv'), index=False)
results_full_data

In [ ]:
results_full_data = pd.read_csv(os.path.join(models_folder, 'full_model_results_summary.csv'))
results_full_data

In [ ]:
results_col = {}
scaler, X_train, y_train, X_val, y_val, X_test, y_test = None, None, None, None, None, None, None
for key in processed_aggs.keys():
    df = processed_aggs[key]
    results_col[key] = {}
    for col in df.columns:
        
        if col == 'index' or col == 'code':
            continue

        results_col[key][col] = {}

        # print(key, col)
        # col_df = prepare_column_for_training(df, col)
        # print(f"Prepared DataFrame for {key} - {col}: {col_df.shape}")
        scaler, X_train, y_train, X_val, y_val, X_test, y_test = prepare_single_channel_for_training(df, col_name=col)
        print(f"Train/Test split for {key} - {col}:")
        print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
        print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

        model_rf, rf_val_accuracy, rf_val_f1 = evaluate_RandomForest_model(
            X_train, y_train, X_val, y_val, X_test, y_test,
            Data_name=f'{key}_{col}', Model_name='RandomForest'
        )

        model_cnn, cnn_val_accuracy, cnn_val_f1 = evaluate_single_channel_dnn_models(
            X_train, y_train, X_val, y_val, X_test, y_test,
            Data_name=f'{key}_{col}', Model_name='BasicSingleChannelCNN'
        )
        results_col[key][col] = {
            "RandomForest": {"acc": rf_val_accuracy, "f1": rf_val_f1},
            "BasicSingleChannelCNN": {"acc": cnn_val_accuracy, "f1": cnn_val_f1}
        }

        with open(os.path.join(models_folder, 'latest_model_results.json'), 'w') as f:
            json.dump(results_col, f, indent=4)

results_col

In [ ]:
y_train

In [ ]:
results_col_data = []

for key in results_col.keys():
    # print(f"Results for {key}:")
    for col in results_col[key].keys():
        # print(f" Channel: {col}")
        # print(f"  RandomForest - Acc: {results_col[key][col]['RandomForest']['acc']}, F1: {results_col[key][col]['RandomForest']['f1']}")
        # print(f"  BasicSingleChannelCNN - Acc: {results_col[key][col]['BasicSingleChannelCNN']['acc']}, F1: {results_col[key][col]['BasicSingleChannelCNN']['f1']}")
        results_col_data.append({
            "Device" : key,
            "Channel" : col,
            "RF_acc" : results_col[key][col]["RandomForest"]["acc"],
            "CNN_acc" : results_col[key][col]["BasicSingleChannelCNN"]["acc"],
            "RF_f1" : results_col[key][col]["RandomForest"]["f1"],
            "CNN_f1" : results_col[key][col]["BasicSingleChannelCNN"]["f1"]
        })
results_col_data = pd.DataFrame(results_col_data)
results_col_data.sort_values(by=["Device", "Channel"]).reset_index(drop=True)
results_col_data.to_csv(os.path.join(models_folder, 'channel_model_results_summary.csv'), index=False)
results_col_data

In [ ]:
results_col_data.aggregate({'RF_acc': ['mean'], 'CNN_acc': ['mean']})

In [ ]:
MU_file = os.path.join(prk_folder, 'MU.parquet')
IN_file = os.path.join(prk_folder, 'IN.parquet')
EP_file = os.path.join(prk_folder, 'EP.parquet')
MW_file = os.path.join(prk_folder, 'MW.parquet')

mw_df = pd.read_parquet(MW_file)
in_df = pd.read_parquet(IN_file)
ep_df = pd.read_parquet(EP_file)
mu_df = pd.read_parquet(MU_file)

In [ ]:
total_samples = (mw_df.shape[0] + in_df.shape[0] +  ep_df.shape[0] + mu_df.shape[0]) * 2
total_samples

In [ ]:
((results_col_data["RF_acc"].mean() + results_col_data["CNN_acc"].mean() ) / 2) * total_samples

In [ ]:
(results_col_data["RF_acc"].mean() + results_col_data["CNN_acc"].mean())  * total_samples / 2

In [ ]:
# --- ML Model Output ---
total_samples = (mw_df.shape[0] + in_df.shape[0] +  ep_df.shape[0] + mu_df.shape[0]) * 2
# correct_predictions =  ((results_col_data["RF_acc"].mean() + results_col_data["CNN_acc"].mean() ) / 2) * total_samples  # Number of successes (count)
# total_samples = results_col_data.shape[0] * 2          # Total trials (nobs)
chance_baseline = 0.10        # H0: Accuracy <= 10%
correct_predictions = 0.56321 * total_samples  # Number of successes (count)
# --- Perform the Z-Test ---
# We test if our model's accuracy is 'larger' than the baseline.
z_stat, p_value = proportions_ztest(count=correct_predictions, nobs=total_samples, value=chance_baseline, alternative='larger')

# --- Interpret the Results ---
model_accuracy = correct_predictions / total_samples
# model_accuracy = 56
print(f"Model Accuracy: {model_accuracy:.4f}")
print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value: {p_value:.4f}")

alpha = 0.05  # Significance level
if p_value < alpha:
    print("\n✅ Result: Reject the null hypothesis.")
    print("Conclusion: The model's accuracy is significantly greater than the 10% chance baseline.")
else:
    print("\n❌ Result: Fail to reject the null hypothesis.")
    print("Conclusion: There is not enough evidence to say the model performs better than chance.")


In [ ]:
tmp_df = prepare_single_channelfor_training(mu_agg_df, col_name='FP2')
tmp_df

In [ ]:
def evaluate_10_crossfold_cnn(df, device = None):
    label_col = 'code'
    n_classes = 10
    df = df[df[label_col] != -1]  # Removing Noise signals

    # 'index' and 'code' are the identifier columns.
    id_vars = ['index', label_col]

    epochs, batch_size = 200, 128

    MW_Model_file = os.path.join(models_folder, 'MW_Full_BasicSingleChannelCNN_CNN.keras')
    MU_Model_file = os.path.join(models_folder, 'MU_Full_best_BasicSingleChannelCNN_weights.keras')
    IN_Model_file = os.path.join(models_folder, 'IN_Full_best_BasicSingleChannelCNN_weights.keras')
    EP_Model_file = os.path.join(models_folder, 'EP_Full_best_BasicSingleChannelCNN_weights.keras')



    models = {
        "MW": MW_Model_file,
        "MU": MU_Model_file,
        "IN": IN_Model_file,
        "EP": EP_Model_file
    }

    model = tf.keras.models.load_model(models[device])
    # All other columns are assumed to be the channel data columns.
    channel_cols = [col for col in df.columns if col not in id_vars]

    concatenated_arrays = df[channel_cols].apply(
        lambda row: np.concatenate(row.values), axis=1
    )

    expanded_df = pd.DataFrame(
        concatenated_arrays.to_list(),
        index=df.index
    )
    
    # Convert all column names to string format
    expanded_df.columns = [str(col) for col in expanded_df.columns]
    complete_df = pd.concat([df[id_vars], expanded_df], axis=1)

    if "index" in complete_df.columns:
        complete_df = complete_df.drop(columns=["index"])


    # print(f"Complete DataFrame shape: {complete_df.shape}")
    # print(f"Complete DataFrame columns: {complete_df.columns.tolist()}")
    if complete_df.empty:
        logging.error("No feature columns (starting with 'data_point_') found in the DataFrame for training.")
        return None, np.array([]), np.array([]), np.array([]), np.array([])

    X = complete_df
    y = df[label_col].copy()
    
    if X.empty or y.empty:
        logging.error("Feature matrix (X) or label vector (y) is empty.")
        return None, np.array([]), np.array([]), np.array([]), np.array([])

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    fold_accuracies = {}
    fold_f1_scores = {}

    for fold, (train_index, test_index) in enumerate(skf.split(X_scaled, y)):
        fold_cnt = fold + 1
        fold_accuracies[fold_cnt] = {}
        fold_f1_scores[fold_cnt] = {}

        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test)

        model_cnn, cnn_val_accuracy, cnn_val_f1 = evaluate_single_channel_dnn_models(
            X_train, y_train, X_val, y_val, X_test, y_test,
            Data_name=f'{device}_Fold_{str(fold_cnt)}', Model_name='BasicSingleChannelCNN'
        )

        # # Reshape for CNN input
        # X_train_c = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        # X_test_c = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
        # y_test_cat = tf.keras.utils.to_categorical(y_test, num_classes=n_classes)

        # print(f"Evaluating model {device} on fold {fold_cnt}...")
        # # Load saved model
        # loss, accuracy = model.evaluate(X_test_c, y_test_cat, batch_size=batch_size, verbose=0)
        # y_pred_prob = model.predict(X_test_c, batch_size=batch_size, verbose=0)
        # y_pred = np.argmax(y_pred_prob, axis=1)

        fold_accuracies[fold_cnt][device] = cnn_val_accuracy
        fold_f1_scores[fold_cnt][device] = cnn_val_f1


    print(f"Fold Accuracies: {fold_accuracies}")
    print(f"Fold F1 Scores: {fold_f1_scores}")
    return fold_accuracies, fold_f1_scores


In [ ]:
result_10_fold = {
    "MW": {},
    "MU": {},
    "IN": {},
    "EP": {}
}

mu_agg_df = pd.read_parquet(MU_Process_Agg_file)
fold_accuracies, fold_f1_scores = evaluate_10_crossfold_cnn(mu_agg_df, device='MU')
result_10_fold["MU"]["Acc"] = fold_accuracies
result_10_fold["MU"]["F1"] = fold_f1_scores
mw_agg_df = None
gc.collect()

with open(os.path.join(models_folder, '10_fold_results.json'), 'w') as f:
    json.dump(result_10_fold, f, indent=4)

mw_agg_df = pd.read_parquet(MW_Process_Agg_file)
fold_accuracies, fold_f1_scores = evaluate_10_crossfold_cnn(mw_agg_df, device='MW')
result_10_fold["MW"]["Acc"] = fold_accuracies
result_10_fold["MW"]["F1"] = fold_f1_scores
mw_agg_df = None
gc.collect()

with open(os.path.join(models_folder, '10_fold_results.json'), 'w') as f:
    json.dump(result_10_fold, f, indent=4)

in_agg_df = pd.read_parquet(IN_Process_Agg_file)
fold_accuracies, fold_f1_scores = evaluate_10_crossfold_cnn(in_agg_df, device='IN')
result_10_fold["IN"]["Acc"] = fold_accuracies
result_10_fold["IN"]["F1"] = fold_f1_scores
in_agg_df = None
gc.collect()

with open(os.path.join(models_folder, '10_fold_results.json'), 'w') as f:
    json.dump(result_10_fold, f, indent=4)

ep_agg_df = pd.read_parquet(EP_Process_Agg_file)
fold_accuracies, fold_f1_scores = evaluate_10_crossfold_cnn(ep_agg_df, device='EP')
result_10_fold["EP"]["Acc"] = fold_accuracies
result_10_fold["EP"]["F1"] = fold_f1_scores
ep_agg_df = None
gc.collect()

with open(os.path.join(models_folder, '10_fold_results.json'), 'w') as f:
    json.dump(result_10_fold, f, indent=4)


In [ ]:
result_10_fold_data = []
result_10_fold_list = {}
for device in result_10_fold.keys():
    # print(f"Device: {device}")
    accs = result_10_fold[device]["Acc"]
    f1s = result_10_fold[device]["F1"]
    result_10_fold_list[device] = {"Acc": [], "F1": []}
    for fold in accs.keys():
        # print(f" Fold {fold}: Acc = {accs[fold][device]:.4f}, F1 = {f1s[fold][device]:.4f}")
        result_10_fold_data.append({
            "Device" : device,
            "Fold" : fold,
            "Acc" : accs[fold][device],
            "F1" : f1s[fold][device]
        })
        result_10_fold_list[device]["Acc"].append(accs[fold][device])
        result_10_fold_list[device]["F1"].append(f1s[fold][device])
result_10_fold_data_df = pd.DataFrame(result_10_fold_data)
result_10_fold_data_df.to_csv(os.path.join(models_folder, '10_fold_model_results_summary.csv'), index=False)
result_10_fold_data_df


In [ ]:
result_10_fold_list

In [ ]:
result_10_fold_list["MW"]["Acc"]

In [ ]:

# --- Perform the One-Way ANOVA Test ---
f_stat, p_value = stats.f_oneway(   result_10_fold_list["MW"]["Acc"], 
                                    result_10_fold_list["MU"]["Acc"], 
                                    result_10_fold_list["IN"]["Acc"], 
                                    result_10_fold_list["EP"]["Acc"])

# --- Interpret the Results ---
print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("\n✅ Result: Reject the null hypothesis.")
    print("Conclusion: At least one device model's mean accuracy is different")
else:
    print("\n❌ Result: Fail to reject the null hypothesis.")
    print("Conclusion:  All four device models have equal mean accuracy")

In [ ]:
all_scores = []
group_labels = []
avg_acc = 0
for device in result_10_fold_list.keys():
    all_scores = all_scores + result_10_fold_list[device]["Acc"]
    mean_acc = np.mean(result_10_fold_list[device]["Acc"])
    avg_acc = avg_acc + mean_acc
    std_acc = np.std(result_10_fold_list[device]["Acc"])
    print(f"{device} - Mean Accuracy: {mean_acc:.4f}, Std Dev: {std_acc:.4f}")

    # Create a list of group labels that corresponds to each score
    group_labels = group_labels + ([device] * len(result_10_fold_list[device]["Acc"]))

all_scores
group_labels
avg_acc / 4


In [ ]:
# # Combine all accuracy scores into one list
# all_scores = np.concatenate([accuracies_mw, accuracies_ep, accuracies_mu, accuracies_in])

# # Create a list of group labels that corresponds to each score
# group_labels = ['MW'] * len(accuracies_mw) + \
#                ['EP'] * len(accuracies_ep) + \
#                ['MU'] * len(accuracies_mu) + \
#                ['IN'] * len(accuracies_in)

# --- 3. Perform the Tukey's HSD test ---
# We set alpha (the significance level) to 0.05.
tukey_results = pairwise_tukeyhsd(endog=all_scores, groups=group_labels, alpha=0.05)

# --- 4. Print and interpret the results ---
print("Tukey's HSD Post-Hoc Test Results:")
print(tukey_results)

print("\n--- Interpretation Guide ---")
print("group1, group2: The pair of devices being compared.")
print("meandiff: The difference between the mean accuracies of the two groups.")
print("p-adj: The adjusted p-value for the comparison. This is the most important value.")
print("reject: 'True' if the null hypothesis for this pair should be rejected (i.e., the difference is statistically significant).")
print("\nConclusion: Look for pairs where 'reject' is 'True' or 'p-adj' is less than 0.05.")


In [ ]:
mw_df.shape, in_df.shape, ep_df.shape, mu_df.shape

In [ ]:
from utilities import preprocess_signal_for_viz, string2array, get_time_vector, band_pass_filter
import plotly.graph_objects as go

In [ ]:
EEG_BANDS = {
    'Delta': (0.5, 4), 'Theta': (4, 8), 'Alpha': (8, 12),
    'Beta': (12, 30), 'Gamma': (30, 40)
}
DESIRED_FS = 128  # Target sampling rate

In [ ]:
tmp_data = string2array(mw_df.iloc[2]["data"])
time_vector = get_time_vector(tmp_data, sampling_rate=512)
tmp_code = mw_df.iloc[2]["code"]

In [ ]:
tmp_normalized = preprocess_signal_for_viz(tmp_data, 512)
tmp_normalized_timevector = get_time_vector(tmp_normalized, 128)

In [ ]:
def plotwave(data, vector, code, title="Waveform"):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=vector, y=data, mode='lines', name='Signal'))
    fig.update_layout(title=f"{title} - Target number: {code}",
                      xaxis_title='Time (s)',
                      yaxis_title='Amplitude',
                      template='plotly_white')
    fig.show()

In [ ]:
# Extract bands and plot
def plot_bands(data, original_fs=128, desired_fs=128, code=4):
    filtered_signals = {}
    for band, (low, high) in EEG_BANDS.items():
        filtered = band_pass_filter(data = data, low_cut = low, high_cut = high, fs = original_fs)
        filtered_signals[band] = filtered
        time_vector = get_time_vector(filtered, desired_fs)
        plotwave(filtered, time_vector, code, title=f"{band} Band")
    return filtered_signals

In [ ]:
# Plot the original signal using plotly use time vector on x-axis
plotwave(tmp_data, time_vector, tmp_code, title="Original Signal")


In [ ]:
plotwave(tmp_normalized, tmp_normalized_timevector, tmp_code, title="Processed Signal")

In [ ]:

len(tmp_normalized)

In [ ]:
plot_bands(tmp_normalized, original_fs=128, desired_fs=128, code=tmp_code)